Notebook Display Options
--

**Display full width**

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

**Do not scroll plot windows**

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

White Island Combination Plot
--

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np

%matplotlib inline

In [ ]:
@np.vectorize
def diverr_np(a, aerr, b, berr):
    '''calculate a/b and resulting error, does not assume error is small compared to value'''
    
    val = float(a)/float(b)
   
    maxnumer = float(a) + float(aerr)
    minnumer = float(a) - float(aerr)
    maxdenom = float(b) + float(berr)
    mindenom = float(b) - float(berr)
    
    valmax = maxnumer/mindenom
    valmin = minnumer/maxdenom
    
    err1 = valmax - val
    err2 = val - valmin
    err = max(err1,err2)
    
    return val, err

** Setup data**

In [ ]:
names = ['dt', 'obs', 'err']
rsamnames = ['dt', 'obs']
kgs2tpd = 86.4  #conversion kg/s to t/d
molrat = 1.4545
csratmax = 200
xsize = 20
ysize = 20

** Airborne gas flux data sets**

In [ ]:
#S02 cospec
url = 'https://fits.geonet.org.nz/observation?typeID=SO2-flux-a&siteID=WI000&methodID=cosp'
s02_cosp = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')

#S02 contouring
url = 'https://fits.geonet.org.nz/observation?typeID=SO2-flux-a&siteID=WI000&methodID=cont'
s02_cont = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')

#S02 flyspec
url = 'https://fits.geonet.org.nz/observation?typeID=SO2-flux-a&siteID=WI000&methodID=flsp'
s02_flsp = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')

#C02 contouring
url = 'https://fits.geonet.org.nz/observation?typeID=CO2-flux-a&siteID=WI000&methodID=cont'
c02_cont = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')

#H2S contouring
url = 'https://fits.geonet.org.nz/observation?typeID=H2S-flux-a&siteID=WI000&methodID=cont'
h2s_cont = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')
h2s_cont['obs'] = h2s_cont['obs'] * 100 #H2S very small, x 100 for plotting
h2s_cont['err'] = h2s_cont['err'] * 100

**Crater lake water chemistry data sets**

In [ ]:
#Calculate TDS as we go

#Al-w
url = 'https://fits.geonet.org.nz/observation?typeID=Al-w&siteID=WI201'
al_w = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')
tds = al_w.copy(deep=True)

#B-w
url = 'https://fits.geonet.org.nz/observation?typeID=B-w&siteID=WI201'
b_w = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')
tds['obs'] += b_w['obs']

#Cl-w
url = 'https://fits.geonet.org.nz/observation?typeID=Cl-w&siteID=WI201'
cl_w = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')
tds['obs'] += cl_w['obs']

#Fe-w
url = 'https://fits.geonet.org.nz/observation?typeID=Fe-w&siteID=WI201'
fe_w = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')
tds['obs'] += fe_w['obs']

#K-w
url = 'https://fits.geonet.org.nz/observation?typeID=K-w&siteID=WI201'
k_w = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')
tds['obs'] += k_w['obs']

#Li-w
url = 'https://fits.geonet.org.nz/observation?typeID=Li-w&siteID=WI201'
li_w = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')
tds['obs'] += li_w['obs']

#Mg-w
url = 'https://fits.geonet.org.nz/observation?typeID=Mg-w&siteID=WI201'
mg_w = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')
tds['obs'] += mg_w['obs']

#Na-w
url = 'https://fits.geonet.org.nz/observation?typeID=Na-w&siteID=WI201'
na_w = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')
tds['obs'] += na_w['obs']

#S04-w
url = 'https://fits.geonet.org.nz/observation?typeID=SO4-w&siteID=WI201'
s04_w = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')
tds['obs'] += s04_w['obs']

**Fumarole zero and 2012 dome temperature data sets**

In [ ]:
#fumarole zero thermocouple
url = 'https://fits.geonet.org.nz/observation?typeID=t&siteID=WI210&methodID=thermcoup'
f0tc = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')

#fumarole zero infrared thermometer
url = 'https://fits.geonet.org.nz/observation?typeID=t&siteID=WI210&methodID=therminfra'
f0ti = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')

#2012 dome fumarole thermocouple
url = 'https://fits.geonet.org.nz/observation?typeID=t&siteID=WI211&methodID=thermcoup'
dftc = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')

#2012 dome fumarole FLIR camera
url = 'https://fits.geonet.org.nz/observation?typeID=t&siteID=WI211&methodID=flir'
dftf = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')


**Lake level data sets**

In [ ]:
#photo comparison
url = 'https://fits.geonet.org.nz/observation?typeID=z&siteID=WI200&methodID=photo'
lakp = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')

#pixel measurement from an image
url = 'https://fits.geonet.org.nz/observation?typeID=z&siteID=WI200&methodID=img'
laki = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')

#distance and vertical angle
url = 'https://fits.geonet.org.nz/observation?typeID=z&siteID=WI200&methodID=vangdist'
laka = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')

**RSAM**

In [ ]:
rsamfile = '/home/sherburn/geonet/my_rsam/WIZ.asc'
rsam = pd.read_csv(rsamfile, names=rsamnames, delim_whitespace=True, parse_dates=True, index_col='dt')
rsam = rsam[(rsam['obs']>0)]

**Levelling, selected sites**

In [ ]:
#Peg B, siteID=WI101
url = 'http://fits.geonet.org.nz/observation?siteID=WI101&typeID=z'
lev101 = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')

#Peg F, siteID=WI105
url = 'http://fits.geonet.org.nz/observation?siteID=WI105&typeID=z'
lev105 = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')

#Peg N, siteID=WI113
url = 'http://fits.geonet.org.nz/observation?siteID=WI113&typeID=z'
lev113 = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')

**Soil gas, selected sites**

In [ ]:
#siteID=WI020
url = 'http://fits.geonet.org.nz/observation?siteID=WI020&typeID=CO2-flux-e'
co2020 = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')

#siteID=WI050
url = 'http://fits.geonet.org.nz/observation?siteID=WI050&typeID=CO2-flux-e'
co2050 = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')

#siteID=WI090
url = 'http://fits.geonet.org.nz/observation?siteID=WI090&typeID=CO2-flux-e'
co2090 = pd.read_csv(url, names=names, skiprows=1, parse_dates=True, index_col='dt')

**Mg/Cl ratio**

In [ ]:
#ensure values at same time so can calculate a ratio
mg =  mg_w[mg_w.index.isin(cl_w.index)]
cl =  cl_w[cl_w.index.isin(mg_w.index)]

mgcl = mg.copy(deep=True)
mgcl['obs'] /= cl['obs']

**Molar ratio CO2/SO2**

In [ ]:
#remove all rows in s02_cont and s02_cosp that have a zero observation value, as this causes a 'divide by zero' error
s02_cont = s02_cont[s02_cont['obs']>0]
s02_cosp = s02_cosp[s02_cosp['obs']>0]

In [ ]:
#ensure values at same time so can calculate a ratio
#with s02 by contoring
c02ct = c02_cont[c02_cont.index.isin(s02_cont.index)]
s02ct = s02_cont[s02_cont.index.isin(c02_cont.index)]
#with s02 by cospec
c02cp = c02_cont[c02_cont.index.isin(s02_cosp.index)]
s02cp = s02_cosp[s02_cosp.index.isin(c02_cont.index)]

In [ ]:
#calculate ratios, accounting for errors
#contouring
molrat_ct = pd.DataFrame()
(molrat_ct['val'], molrat_ct['err']) = diverr_np(c02ct['obs'], c02ct['err'], s02ct['obs'], s02ct['err'])
#cospec
molrat_cp = pd.DataFrame()
(molrat_cp['val'], molrat_cp['err']) = diverr_np(c02cp['obs'], c02cp['err'], s02cp['obs'], s02cp['err'])
#make ratios molar, add datetime index
molrat_ct *= molrat
molrat_ct.index = c02ct.index
molrat_cp *= molrat
molrat_cp.index = c02cp.index

**Plot all data**

In [ ]:
fig, (axlv, axll, ax0, ax1, axml, axsl, ax2, axrsam) = plt.subplots(nrows=8,ncols=1, sharex=True, figsize=(xsize, ysize))

datemin = datetime.datetime.strptime('2003-01-01', "%Y-%m-%d")
now = datetime.datetime.now()

#-----------------------------------

#crater floor levelling, relative to a peg close to the sea
lev101['obs'].plot(ax=axlv, marker='o', linestyle=':', color='red', yerr=lev101['err'], label='WI101 (B)')
lev105['obs'].plot(ax=axlv, marker='o', linestyle=':', color='green', yerr=lev105['err'], label='WI105 (F)')
lev113['obs'].plot(ax=axlv, marker='o', linestyle=':', color='blue', yerr=lev113['err'], label='WI113 (N)')

axlv.set_ylabel('height change (m)')
axlv.grid(b=True, which='major', color='black', linestyle='--', alpha=0.5)
axlv.legend(loc = 'best', fontsize = 8)
axlv.set_title('Levelling, selected sites', loc='left')

#-----------------------------------
#lake level, relative to sea level
lakp['obs'].plot(ax=axll, marker='o', linestyle=':', color='red', yerr=lakp['err'], label='from photo')
laki['obs'].plot(ax=axll, marker='o', linestyle=':', color='blue', yerr=laki['err'], label='from pixels')
laka['obs'].plot(ax=axll, marker='o', linestyle=':', color='green', yerr=laka['err'], label='vertical angle')

axll.set_ylabel('lake level (m asl)')
axll.grid(b=True, which='major', color='black', linestyle='--', alpha=0.5)
axll.legend(loc = 'best', fontsize = 8)
axll.set_title('Lake level', loc='left')

#-----------------------------------

#TDS & Mg/Cl
tds['obs'].plot(ax=ax0, marker='o', linestyle=':', color='red', label='TDS')

ax0.set_ylabel('tot disolve sols - TDS (mg/L)', color='red')
ax0.tick_params(axis='y', colors='red')
ax0.grid(b=True, which='major', color='black', linestyle='--', alpha=0.5)
ax0.legend(loc = 'upper left', fontsize = 8)

#alternate y-axis, Mg/Cl
ax0a = ax0.twinx()
mgcl['obs'].plot(ax=ax0a, marker='o', linestyle=':', color='blue', label='Mg/Cl')
ax0a.tick_params(axis='y', colors='blue')
ax0a.set_ylabel('Mg/Cl ratio', color = 'blue')
ax0a.legend(loc = 'upper right', fontsize = 8)
ax0.set_title('TDS & Mg/Cl ratio', loc='left')

#-----------------------------------

#Airborne gas flux
#limited y-axis, drop one large outlier
s02_cosp['obs'].plot(ax=ax1, marker='o', linestyle=':', color='red', yerr=s02_cosp['err'], label='SO2-flux-a cosp')
s02_cont['obs'].plot(ax=ax1, marker='^', linestyle=':', color='red', yerr=s02_cont['err'], label='SO2-flux-a cont')
c02_cont['obs'].plot(ax=ax1, figsize=(xsize, ysize), marker='o', linestyle=':', color='green', yerr=c02_cont['err'], label='CO2-flux-a cont')
h2s_cont['obs'].plot(ax=ax1, figsize=(xsize, ysize), marker='o', linestyle=':', color='blue', yerr=h2s_cont['err'], label='H2S-flux-a cont x 100')

ax1.set_ylabel('gas flux (kg/s)')
ax1.set_ylim(0,50)
ax1.grid(b=True, which='major', color='black', linestyle='--', alpha=0.5)
ax1.legend(loc = 'best', fontsize = 8)

#alternate y-axis, tonnes/day
ax1a = ax1.twinx()
y1,y2 = ax1.get_ylim()
ax1a.set_ylim(y1*kgs2tpd, y2*kgs2tpd)
ax1a.tick_params(axis='y', colors='gray')
ax1a.set_ylabel('gas flux (t/d)', color = 'gray')
ax1.set_title('Airborne gas flux', loc='left')

#-----------------------------------

#Molar ratios, CO2/SO2
molrat_ct['val'].plot(ax=axml, marker='o', linestyle=':', color='blue', yerr=molrat_ct['err'], label='CO2/SO2-cont')
molrat_cp['val'].plot(ax=axml, marker='o', linestyle=':', color='red', yerr=molrat_cp['err'],label='CO2/SO2-cosp')

axml.set_ylabel('C/S molar ratio')
axml.grid(b=True, which='major', color='black', linestyle='--', alpha=0.5)
axml.legend(loc = 'best', fontsize = 8)
axml.set_ylim(0,80)
axml.set_title('C/S molar ratio', loc='left')
               
#-----------------------------------
               
#Soil gas
co2020['obs'].plot(ax=axsl, marker='o', linestyle=':', color='red', yerr=co2020['err'], label='WI020')
co2050['obs'].plot(ax=axsl, marker='o', linestyle=':', color='green', yerr=co2050['err'], label='WI050')
co2090['obs'].plot(ax=axsl, marker='o', linestyle=':', color='blue', yerr=co2090['err'], label='WI090')

axsl.set_ylabel('Flux g/m$^2$/d')
axsl.grid(b=True, which='major', color='black', linestyle='--', alpha=0.5)
axsl.legend(loc = 'best', fontsize = 8)

axsl.set_title('Soil gas, CO2', loc='left')
#-----------------------------------

#Fumarole zero temperatures
f0tc['obs'].plot(ax=ax2, marker='o', linestyle=':', color='red', yerr=f0tc['err'], label='F0 thermcoup')
f0ti['obs'].plot(ax=ax2, marker='o', linestyle=':', color='green', yerr=f0ti['err'], label='F0 therminfra')

ax2.set_ylabel('fumarole zero ($^\circ$C)')
ax2.grid(b=True, which='major', color='black', linestyle='--', alpha=0.5)
ax2.legend(loc = 'upper left', fontsize = 8)

#alternate y-axis, 2012 Dome fumarole temperatures
ax2a = ax2.twinx()
dftc['obs'].plot(ax=ax2a, marker='o', linestyle=':', color='blue', yerr=dftc['err'], label='dome thermcoup')
dftf['obs'].plot(ax=ax2a, marker='o', linestyle=':', color='black', yerr=dftf['err'], label='dome flir')
ax2a.tick_params(axis='y', colors='blue')
ax2a.set_ylabel('dome fumarole ($^\circ$C)', color = 'blue')
ax2a.legend(loc = 'upper right', fontsize = 8)
ax2.set_title('Fumarole temperatures', loc='left')

#-----------------------------------

#WIZ rsam
rsam['obs'].plot(ax=axrsam, marker='None', label='RSAM')

axrsam.set_ylabel('ground velocity (nm/s)')
axrsam.grid(b=True, which='major', color='black', linestyle='--', alpha=0.5)
axrsam.legend(loc = 'best', fontsize = 8)
axrsam.set_title('Daily RSAM', loc='left')

axrsam.set_xlim(datemin, now)